# Tarea 1 Implementación mlp

## Alumno: Gerardo de Miguel González

### Dataset Circular

In [1]:
#::GMG::Cargo los datos del problema no lineal (nube de puntos ejemplo)
circle <- read.csv('data/circle.csv', header = F)

In [2]:
str(circle)

'data.frame':	100 obs. of  3 variables:
 $ V1: num  -0.9719 -0.553 -0.0949 -0.8096 0.399 ...
 $ V2: num  0.847 -0.577 -0.299 0.857 0.69 ...
 $ V3: int  0 0 1 0 0 0 0 0 0 0 ...


In [3]:
#::GMG::Obtengo los índices de los puntos de una de las clases
ind_c <- which(circle[,3]==0)

In [4]:
#::GMG::Dibujo los puntos distinguiendo la clase por color
#::nota:: par(mar = ...)
# A numerical vector of the form c(bottom, left, top, right) which gives 
# the number of lines of margin to be specified on the four sides of the plot. 
# The default is c(5, 4, 4, 2) + 0.1
# https://www.rdocumentation.org/packages/graphics/versions/3.5.3/topics/par
par(mar = c(5,5,1,1))
plot(circle[ind_c,1],circle[ind_c,2],
     type='p',
     xlim=c(-1,1),ylim=c(-1,1), 
     xlab = 'x', ylab = 'y',
     col = 'green')
lines(circle[-ind_c,1],circle[-ind_c,2],
      type='p',col='blue')

### Generalización (capas ocultas)

```r
lin<-read.csv('circle.csv',header= F)
a<-as.matrix(lin[,-3])
b<-as.matrix(lin[,3])
backprop_mlp(a,b, h=5, epochs= 500, eta = 0.1)  
# h: hidden neurons
```

Generalizar la función `backprop_mlp` anterior para que contemple la inclusión de una capa oculta. Aplicar la función al ejemplo de la clasificación circular. 

In [5]:
#::GMG::Elijo la función de activación que voy a aplicar 
#       a las neuronas ocultas y de salida (S8-S10, S01-Intro-FeedForward.pdf)
#::nota:: f'(x) = f(x) (1 - f(x)) (S10, S01-Intro-FeedForward.pdf)
sigmoid <- function(z) {
    1/(1 + exp(-z))
}

In [6]:
#::GMG::Programo Gradient Descent con backpropagation básico
backprop_mlp <- function(y, x, h = 5, epochs = 500, eta = 0.1) {
    ### Inicializar matrices y listas
    #::GMG::Convierto la entrada y salida en matrices
    Y <- as.matrix(y)
    X <- as.matrix(x)
    #::GMG::Inicialización aleatoria de martices de pesos y bias
    w_h <- matrix(data = runif(ncol(X)*h, min = -1, max = 1),
                  nrow = ncol(X), 
                  ncol = h
                 )
    b_h <- matrix(data = rep(runif(h), nrow(X)), 
                  nrow = nrow(X), 
                  byrow = FALSE
                 )    
    w_o <- matrix(data = runif(h*ncol(Y), min = -1, max = 1),
                  nrow = h, 
                  ncol = ncol(Y)
                 )
    b_o <- matrix(data = rep(runif(ncol(Y)), nrow(X)),
                  nrow = nrow(X),
                  byrow = FALSE
                 )
    
    for (j in 1:epochs) {
        #::GMG::FeedForward (propagar hacia delante)
        H <- sigmoid(X%*%w_h + b_h)    
        O <- sigmoid(H%*%w_o + b_o)        

        #::GMG::Back propagation (actualizar pesos)
        error_o <- Y - O
        aux_o <- error_o * (O * (1 - O))
        
        error_h <- aux_o%*%t(w_o)
        aux_h <- error_h * (H * (1 - H))
    
        w_o <- w_o + (t(H)%*%aux_o)*eta
        b_o <- b_o + rowSums(aux_o)*eta

        w_h <- w_h + (t(X)%*%aux_h)*eta
        b_h <- b_h + rowSums(aux_h)*eta        
    }
    
    return(list(error = (t(error_o)%*%error_o)/2, 
                w_h = w_h, b_h = b_h, 
                w_o = w_o, b_o = b_o,
                output = O))
}

### Prueba de concepto

In [7]:
# https://www.analyticsvidhya.com/blog/2017/05/neural-network-from-scratch-in-python-and-r/
# input matrix
# https://www.rdocumentation.org/packages/base/versions/3.5.3/topics/matrix
X <- matrix(c(1,0,1,0,1,0,1,1,0,1,0,1),
            nrow = 3, 
            ncol = 4,
            byrow = TRUEº)
X

1,0,1,0
1,0,1,1
0,1,0,1


In [8]:
# output matrix
Y <- matrix(c(1,1,0),
            byrow=FALSE)
Y

1
1
0


In [9]:
modelo <- backprop_mlp(y = Y, x = X, h = 3, epochs = 5000, eta = 0.1)

In [10]:
modelo$error

0.000955954


In [11]:
modelo$output

0.97952471
0.97848136
0.03208767


Si se fija el número máximo de épocas en 1000. ¿Qué número de neuronas ocultas y qué valor de la tasa de aprendizaje (eta) es óptimo para este problema? (basta una solución aproximada).

### BONUS 1 ¿Sabrías incluir un término de inercia en el método de backpropagation?

NOTA: ver transparencia 4

### BONUS 2 Generalizar a un número arbitrario de capas

Recomendación: definir los pesos como una lista de matrices, una para cada capa

### ::GMG:: Notas

In [15]:
# https://www.analyticsvidhya.com/blog/2017/05/neural-network-from-scratch-in-python-and-r/
# input matrix
# https://www.rdocumentation.org/packages/base/versions/3.5.3/topics/matrix
X <- matrix(c(1,0,1,0,1,0,1,1,0,1,0,1),
            nrow = 3, 
            ncol = 4,
            byrow = TRUE)
X

1,0,1,0
1,0,1,1
0,1,0,1


In [16]:
# output matrix
Y <- matrix(c(1,1,0),
            byrow=FALSE)
Y

1
1
0


In [17]:
#sigmoid function
sigmoid<-function(x){
  1/(1+exp(-x))
}

In [18]:
# derivative of sigmoid function
derivatives_sigmoid<-function(x){
  x*(1-x)
}

In [19]:
# variable initialization
epoch <- 5000
lr <- 0.1
inputlayer_neurons <- ncol(X)
hiddenlayer_neurons <- 3
output_neurons <- 1

In [20]:
#weight and bias initialization
# https://stat.ethz.ch/R-manual/R-devel/library/stats/html/Normal.html
wh <- matrix(rnorm(inputlayer_neurons*hiddenlayer_neurons,
                   mean = 0,
                   sd = 1), 
             inputlayer_neurons, 
             hiddenlayer_neurons
            )

# https://stat.ethz.ch/R-manual/R-patched/library/stats/html/Uniform.html
# https://www.rdocumentation.org/packages/base/versions/3.5.3/topics/rep
bias_in <- runif(hiddenlayer_neurons)
bias_in_temp <- rep(bias_in, nrow(X))

bh <- matrix(bias_in_temp, 
             nrow = nrow(X), 
             byrow = FALSE)

In [21]:
wout <- matrix(rnorm(hiddenlayer_neurons*output_neurons,
                   mean = 0,
                   sd = 1), 
            hiddenlayer_neurons, 
            output_neurons
           )

bias_out <- runif(output_neurons)
bias_out_temp <- rep(bias_out,nrow(X))

bout <- matrix(bias_out_temp,
               nrow = nrow(X),
               byrow = FALSE)

In [22]:
for(i in 1:epoch){
    # forward propagation
    hidden_layer_input1 <- X%*%wh
    hidden_layer_input <- hidden_layer_input1 + bh
    hidden_layer_activations <- sigmoid(hidden_layer_input)
    
    output_layer_input1 <- hidden_layer_activations%*%wout
    output_layer_input <- output_layer_input1 + bout
    output <- sigmoid(output_layer_input)
    
    # Back Propagation
    E <- Y - output
    slope_output_layer <- derivatives_sigmoid(output)
    d_output <- E*slope_output_layer
    
    Error_at_hidden_layer <- d_output%*%t(wout)
    slope_hidden_layer <- derivatives_sigmoid(hidden_layer_activations)
    d_hiddenlayer <- Error_at_hidden_layer*slope_hidden_layer
    
    wout <- wout + (t(hidden_layer_activations)%*%d_output)*lr
    bout <- bout + rowSums(d_output)*lr

    wh <- wh + (t(X)%*%d_hiddenlayer)*lr
    bh <- bh + rowSums(d_hiddenlayer)*lr
}

In [23]:
t(E)%*%E/2

0.0006325006


In [24]:
output

0.98351371
0.98145359
0.02548007


In [25]:
E

0.01648629
0.01854641
-0.02548007


In [26]:
wh

-1.40633981,-0.05063875,1.6164089
1.78520772,-2.69617713,0.5345753
-0.51391954,1.85818010,0.8376028
-0.03829498,0.23579005,-2.2490879


In [27]:
bh

0.2012972,0.2012972,0.2012972
0.1606424,0.1606424,0.1606424
0.4155168,0.4155168,0.4155168


In [28]:
wout

-2.581353
1.844152
1.642608


In [29]:
bout

1.320312
1.705068
-1.892493
